In [1]:
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

In [2]:
f= open('../datasets/shakespeare.txt','r')
d=str(f.read())
f.close()

In [3]:
data= d.strip().lower()
data=data[104:]

In [10]:
print data[5000:5100]

 a refund of the money (if any) you paid
for it by sending an explanatory note within that time to 


In [5]:
chars=sorted(list(set(data)))
print len(chars)

66


In [7]:
char_indices=dict((c,i) for i,c in enumerate(chars))
indices_char=dict((i,c) for i,c in enumerate(chars))
print len(char_indices),len(indices_char)

66 66


In [15]:
sent=data.split('\n')

In [25]:
lengths=[]
for ix in sent:
    lengths.append(len(ix))
print len(lengths)

124784


In [26]:
uniq, nums= np.unique(np.array(lengths),return_counts=True)

In [28]:
print uniq[:10],nums[:10]

 [ 1  5  6  7  8  9 10 11 12 13] [9681   43   77   33  103  264  451  407  278  273]


In [30]:
maxlen=50
step=3
sentences=[]
next_chars=[]
for i in range(0,len(data)-maxlen,step):
    sentences.append(data[i:i+maxlen])
    next_chars.append(data[i+maxlen])
print ('nb sequences: ' , len(sentences))

('nb sequences: ', 1863245)


In [34]:
print sentences[100:110]

['ook, details below **\r\n**     please follow the co', ', details below **\r\n**     please follow the copyr', 'etails below **\r\n**     please follow the copyrigh', 'ils below **\r\n**     please follow the copyright g', ' below **\r\n**     please follow the copyright guid', 'low **\r\n**     please follow the copyright guideli', ' **\r\n**     please follow the copyright guidelines', '\r\n**     please follow the copyright guidelines in', '*     please follow the copyright guidelines in th', '   please follow the copyright guidelines in this ']


In [33]:
print ('Vectorization...')
x= np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y=np.zeros((len(sentences),len(chars)),dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentences):
        x[i,t, char_indices[char]]=1
    y[i,char_indices[next_chars[i]]]=1

Vectorization...


KeyError: 'this ebook is for the use of anyone anywhere at no'